# Python Resource Management Project:

[Overview link](https://console.cloud.google.com/auth/overview?project=python-resource-mgt)

<br>

GCP Service account:  ```moneypenny@python-resource-mgt.iam.gserviceaccount.com```


### OAuth Scopes:

* Gmail:
    * ```https://www.googleapis.com/auth/gmail.modify```
    * ```https://www.googleapis.com/auth/gmail.labels```

* Sheets:
    * ```https://www.googleapis.com/auth/spreadsheets```

* Drive:
    * ```https://www.googleapis.com/auth/drive.file```

In [ ]:
# requirements.txt

google-auth-oauthlib==1.0.0
google-auth-httplib2==0.1.0
google-api-python-client==2.86.0
pandas==2.0.0
numpy==1.24.0
matplotlib==3.7.0
seaborn==0.12.0
schedule==1.2.0
pytz==2023.3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import base64
import email
import pytz
import schedule
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import base64
import pytz
import schedule
import time
import pickle

SCOPES = [
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/gmail.labels',
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive.file'
]

class FinancialEmailProcessor:
    def __init__(self, credentials):
        self.gmail_service = build('gmail', 'v1', credentials=credentials)
        self.sheets_service = build('sheets', 'v4', credentials=credentials)
        self.processed_ids_file = 'processed_email_ids.txt'
        self.user_id = 'me'
        self.label_id = self.get_or_create_label('Portfolio')

    def get_or_create_label(self, label_name):
        """Get or create Gmail label"""
        try:
            results = self.gmail_service.users().labels().list(userId=self.user_id).execute()
            labels = results.get('labels', [])
            
            # Check if label exists
            for label in labels:
                if label['name'] == label_name:
                    return label['id']
            
            # Create new label if it doesn't exist
            label_object = {
                'name': label_name,
                'labelListVisibility': 'labelShow',
                'messageListVisibility': 'show'
            }
            created_label = self.gmail_service.users().labels().create(
                userId=self.user_id,
                body=label_object
            ).execute()
            return created_label['id']
            
        except Exception as e:
            print(f"Error with label: {e}")
            return None

    def get_financial_emails(self):
        """Get financial notification emails"""
        processed_ids = self.load_processed_ids()
        # Customize senders list as needed
        senders = ['etrade.com', 'chase.com', 'schwab.com']  
        query = f"from:({' OR '.join(senders)})"
        
        results = self.gmail_service.users().messages().list(
            userId=self.user_id, q=query).execute()
        messages = results.get('messages', [])
        
        new_transactions = []
        
        for message in messages:
            if message['id'] not in processed_ids:
                msg = self.gmail_service.users().messages().get(
                    userId=self.user_id, id=message['id'], format='full').execute()
                
                # Add Portfolio label
                if self.label_id:
                    self.gmail_service.users().messages().modify(
                        userId=self.user_id,
                        id=message['id'],
                        body={'addLabelIds': [self.label_id]}
                    ).execute()
                
                # Extract transaction details
                transaction = self.parse_email(msg)
                if transaction:
                    new_transactions.append(transaction)
                
                self.save_processed_id(message['id'])
        
        return pd.DataFrame(new_transactions) if new_transactions else pd.DataFrame()

def get_credentials():
    """Get and refresh OAuth credentials"""
    creds = None
    
    # Load saved credentials if they exist
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # Refresh if expired
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    # Otherwise, let user log in
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret.json',  # Your OAuth client secrets file
            SCOPES,
            redirect_uri='http://localhost:8080'  # We'll modify this for WSL
        )
        creds = flow.run_local_server(port=8080)
        
        # Save credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return creds

def main():
    credentials = get_credentials()
    processor = FinancialEmailProcessor(credentials)
    SPREADSHEET_ID = 'your_spreadsheet_id'
    
    # Rest of your main function...

if __name__ == '__main__':
    main()